In [33]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import os
from package import key, storage
from package.osm import osm
from package.logger import Timed, rlog
import pandas as pd
import folium

In [35]:
from package.logger import setup
setup("INFO")

In [39]:
city_id = "Koeln"
stops_path = "../data/cleaned/stops.csv"
osm_path = osm.get_osm_path_from_city_id(city_id)

with Timed.info("Reading stops"):
	stops_df = storage.read_gdf(stops_path)

if not os.path.exists(osm_path) and city_id:
	rlog.info("Downloading OSM data")
	osm.download_city(city_id, osm_path)
else:
	rlog.info("Using existing OSM data")

osm_reader = osm.new_osm_reader(osm_path)

with Timed.info("Getting OSM graph"):
	nodes, edges = osm.get_graph_for_city_cropped_to_stops(osm_reader, stops_df)


[11:24:31] INFO     Reading stops                                                                   ]8;id=727497;file:///tmp/ipykernel_33631/2164838183.py\2164838183.py]8;;\:]8;id=683893;file:///tmp/ipykernel_33631/2164838183.py#5\5]8;;\

           INFO     Reading stops done (0.03 seconds)                                               ]8;id=645019;file:///tmp/ipykernel_33631/2164838183.py\2164838183.py]8;;\:]8;id=901227;file:///tmp/ipykernel_33631/2164838183.py#5\5]8;;\

           INFO     Using existing OSM data                                                        ]8;id=739452;file:///tmp/ipykernel_33631/2164838183.py\2164838183.py]8;;\:]8;id=628322;file:///tmp/ipykernel_33631/2164838183.py#12\12]8;;\

           INFO     Getting OSM graph                                                              ]8;id=73236;file:///tmp/ipykernel_33631/2164838183.py\2164838183.py]8;;\:]8;id=359677;file:///tmp/ipykernel_33631/2164838183.py#16\16]8;;\

           INFO     Hash for OSM network:                                                          ]8;id=132653;file:///tmp/ipykernel_33631/2164838183.py\2164838183.py]8;;\:]8;id=659172;file:///tmp/ipykernel_33631/2164838183.py#16\16]8;;\
                    25519526578071114144969288949484232041135192474713375411543851787353293669047                  

           INFO     Loading OSM network from cache                                                 ]8;id=969510;file:///tmp/ipykernel_33631/2164838183.py\2164838183.py]8;;\:]8;id=950454;file:///tmp/ipykernel_33631/2164838183.py#16\16]8;;\

[11:24:32] INFO     Getting OSM graph done (0.43 seconds)                                          ]8;id=845092;file:///tmp/ipykernel_33631/2164838183.py\2164838183.py]8;;\:]8;id=876553;file:///tmp/ipykernel_33631/2164838183.py#16\16]8;;\

In [5]:
# edges['lit'] = edges['lit'].apply(lambda x: 1 if x == 'yes' else 0)
# edges['not_lit'] = edges['lit'].apply(lambda x: 0 if x == 1 else 1)
# edges['length_not_lit'] = (edges['length'] * edges['not_lit']).astype(int)

# edges['length'] = (edges['length'] * 1000).astype(int)

# edges['weights'] = '(' + edges['length'].astype(str) + ';' + edges['length_not_lit'].astype(str) + ')'

# edges[['u', 'v', 'weights']].to_csv("../data/mlc_edges.csv", index=False)

### multi modal

In [37]:
nodes["has_bicycle"] = False
nodes.loc[nodes.sample(100).index, "has_bicycle"] = True
nodes = nodes[['id', 'has_bicycle']]

In [7]:
walking_nodes = nodes.copy()
bike_nodes = nodes.copy()

walking_nodes["old_id"] = walking_nodes["id"]
walking_nodes["id"] = "W" + walking_nodes["id"].astype(str)

bike_nodes["old_id"] = bike_nodes["id"]
bike_nodes["id"] = "B" + bike_nodes["id"].astype(str)

In [8]:
edges = edges[["u", "v", "length"]]
walking_edges = edges.copy()
bike_edges = edges.copy()

walking_edges["u"] = "W" + walking_edges["u"].astype(str)
walking_edges["v"] = "W" + walking_edges["v"].astype(str)

bike_edges["u"] = "B" + bike_edges["u"].astype(str)
bike_edges["v"] = "B" + bike_edges["v"].astype(str)

In [9]:
# create edges that go from each bicycle node to each walking node - free floating bicycle system
transfer_edges_values = walking_nodes.apply(
    lambda x: ["B" + str(x.old_id), "W" + str(x.old_id), 0], axis=1
)
transfer_edges = pd.DataFrame(
    transfer_edges_values.tolist(), columns=["u", "v", "length"]
)

In [10]:
avg_walking_speed = 1.4 # m/s
avg_biking_speed = 4.0 # m/s

In [11]:
walking_edges["travel_time"] = walking_edges.length / avg_walking_speed
bike_edges["travel_time"] = bike_edges.length / avg_biking_speed

In [12]:
walking_edges

,u,v,length,travel_time
0,W21063145,W7151289920,7.161,5.115000
1,W7151289920,W10929975,7.575,5.410714
2,W10929975,W10853912,132.615,94.725000
4,W367149,W1672380543,7.730,5.521429
5,W1672380543,W1588847134,11.032,7.880000
...,...,...,...,...
1331671,W11105976044,W11105976043,3.237,2.312143
1331672,W11105943446,W11105943462,13.513,9.652143
1331673,W11105943462,W11105943463,9.101,6.500714
1331674,W11105943463,W11105943464,12.710,9.078571


In [13]:
bike_edges["travel_time_bike"] = bike_edges["travel_time"]

In [14]:
edges = pd.concat([walking_edges, bike_edges, transfer_edges], ignore_index=True)
# fill travel_time for transfer edges and
# travel_time_bike for walking and transfer edges
edges = edges.fillna(0)
# weights = trave_time, travel_time_bike, cost
accuracy = 4
accuracy_multiplier = 10 ** (accuracy - 1)
print(accuracy_multiplier)
edges["weights"] = (
    "("
    + (edges["travel_time"].round(accuracy) * accuracy_multiplier)
    .astype(int)
    .astype(str)
    + ";0)"
)
edges["hidden_weights"] = (
    "("
    + (edges["travel_time_bike"].round(accuracy) * accuracy_multiplier)
    .astype(int)
    .astype(str)
    + ")"
)
edges.head(2)


1000


,u,v,length,travel_time,travel_time_bike,weights,hidden_weights
0,W21063145,W7151289920,7.161,5.115000,0.0,(5115;0),(0)
1,W7151289920,W10929975,7.575,5.410714,0.0,(5410;0),(0)


In [15]:
nodes = pd.concat([walking_nodes, bike_nodes])
nodes.head(2)

,id,has_bicycle,old_id
0,W21063145,False,21063145
1,W7151289920,False,7151289920


In [16]:
node_map = {}
for i, node_id in enumerate(nodes.id.unique()):
	node_map[node_id] = i

nodes["old_id"] = nodes["id"]
nodes["id"] = nodes["id"].map(node_map)
edges["u"] = edges["u"].map(node_map)
edges["v"] = edges["v"].map(node_map)

In [17]:
walking_node_map = {}
for i, node_id in enumerate(walking_nodes.id.unique()):
	walking_node_map[node_id] = i

walking_nodes["old_id"] = walking_nodes["id"]
walking_nodes["id"] = walking_nodes["id"].map(node_map)
walking_edges["u"] = walking_edges["u"].map(node_map)
walking_edges["v"] = walking_edges["v"].map(node_map)

In [18]:
print(nodes.id.isna().sum())
print(edges.u.isna().sum())
print(edges.v.isna().sum())

print(walking_nodes.id.isna().sum())
print(walking_edges.u.isna().sum())
print(walking_edges.v.isna().sum())

0
0
0
0
0
0


In [19]:
edges.sort_values('travel_time_bike')

,u,v,length,travel_time,travel_time_bike,weights,hidden_weights
0,0,1,7.161,5.11500,0.00000,(5115;0),(0)
135494,68638,15454,0.000,0.00000,0.00000,(0;0),(0)
135493,68637,15453,0.000,0.00000,0.00000,(0;0),(0)
135492,68636,15452,0.000,0.00000,0.00000,(0;0),(0)
135491,68635,15451,0.000,0.00000,0.00000,(0;0),(0)
...,...,...,...,...,...,...,...
103428,78087,77808,414.769,103.69225,103.69225,(103692;0),(103692)
112629,100751,77799,414.769,103.69225,103.69225,(103692;0),(103692)
61734,55096,55097,488.963,122.24075,122.24075,(122240;0),(122240)
107677,96725,62234,539.549,134.88725,134.88725,(134887;0),(134887)


In [20]:
edges[["u", "v", "weights", "hidden_weights"]].to_csv(
    "../data/mlc_edges.csv", index=False
)

In [21]:
walking_edges["weights"] = (
    "("
    + (walking_edges["travel_time"].round(accuracy) * accuracy_multiplier)
    .astype(int)
    .astype(str)
    + ")"
)


In [22]:
walking_edges[["u", "v", "weights"]].to_csv(
    "../data/mlc_walking_edges.csv", index=False
)

In [23]:
storage.write_any_dict(
    {"node_map": node_map, "walking_node_map": walking_node_map}, "../data/node_map.pkl"
)


In [24]:
start_node_id = walking_node_map["W21063145"]
start_bags = {
	start_node_id: {
		"node_id": start_node_id,
		"path": [start_node_id],
		"values": [0],
	}
}

In [25]:
import mcr_py
from mcr_py import GraphCache

In [26]:
raw_edges = walking_edges[["u", "v", "weights"]].to_dict("records")

gc = GraphCache()
gc.set_graph(raw_edges)

In [27]:
gc.summary()

[10:38:58] INFO     Nodes: 53184                                                                  ]8;id=668869;file://src/rs/graph_cache.rs\graph_cache.rs]8;;\:]8;id=348630;file://src/rs/graph_cache.rs#30\30]8;;\

           INFO     Edges: 60020                                                                  ]8;id=556861;file://src/rs/graph_cache.rs\graph_cache.rs]8;;\:]8;id=659197;file://src/rs/graph_cache.rs#31\31]8;;\

In [28]:
bags = mcr_py.run_mlc(gc, 0)

           INFO     queue size: 90                                                                       ]8;id=744189;file:///home/moritz/dev/uni/mlc/src/mlc.rs\mlc.rs]8;;\:]8;id=725343;file:///home/moritz/dev/uni/mlc/src/mlc.rs#215\215]8;;\

           INFO     queue size: 122                                                                      ]8;id=178051;file:///home/moritz/dev/uni/mlc/src/mlc.rs\mlc.rs]8;;\:]8;id=774195;file:///home/moritz/dev/uni/mlc/src/mlc.rs#215\215]8;;\

           INFO     queue size: 122                                                                      ]8;id=909152;file:///home/moritz/dev/uni/mlc/src/mlc.rs\mlc.rs]8;;\:]8;id=573791;file:///home/moritz/dev/uni/mlc/src/mlc.rs#215\215]8;;\

           INFO     queue size: 134                                                                      ]8;id=142266;file:///home/moritz/dev/uni/mlc/src/mlc.rs\mlc.rs]8;;\:]8;id=212657;file:///home/moritz/dev/uni/mlc/src/mlc.rs#215\215]8;;\

           INFO     queue size: 172                                                                      ]8;id=8893;file:///home/moritz/dev/uni/mlc/src/mlc.rs\mlc.rs]8;;\:]8;id=370932;file:///home/moritz/dev/uni/mlc/src/mlc.rs#215\215]8;;\

...

           INFO     queue size: 125                                                                      ]8;id=621608;file:///home/moritz/dev/uni/mlc/src/mlc.rs\mlc.rs]8;;\:]8;id=815508;file:///home/moritz/dev/uni/mlc/src/mlc.rs#215\215]8;;\

           INFO     queue size: 103                                                                      ]8;id=510620;file:///home/moritz/dev/uni/mlc/src/mlc.rs\mlc.rs]8;;\:]8;id=960431;file:///home/moritz/dev/uni/mlc/src/mlc.rs#215\215]8;;\

           INFO     queue size: 89                                                                       ]8;id=536687;file:///home/moritz/dev/uni/mlc/src/mlc.rs\mlc.rs]8;;\:]8;id=932309;file:///home/moritz/dev/uni/mlc/src/mlc.rs#215\215]8;;\

           INFO     queue size: 69                                                                       ]8;id=757017;file:///home/moritz/dev/uni/mlc/src/mlc.rs\mlc.rs]8;;\:]8;id=903949;file:///home/moritz/dev/uni/mlc/src/mlc.rs#215\215]8;;\

           INFO     queue size: 19                                                                       ]8;id=969874;file:///home/moritz/dev/uni/mlc/src/mlc.rs\mlc.rs]8;;\:]8;id=890672;file:///home/moritz/dev/uni/mlc/src/mlc.rs#215\215]8;;\

In [44]:
bags

{13597: [<PyLabel at 0x7fd9b46c28e0>],
 40849: [<PyLabel at 0x7fd9b3f74110>],
 29539: [<PyLabel at 0x7fd9b3ea3600>],
 7602: [<PyLabel at 0x7fd9b43ccce0>],
 7172: [<PyLabel at 0x7fd9b464b130>],
 14322: [<PyLabel at 0x7fd9b45c0180>],
 48504: [<PyLabel at 0x7fd9b3ebcd50>],
 38913: [<PyLabel at 0x7fd9b42a6fe0>],
 24531: [<PyLabel at 0x7fd9b3faa560>],
 12091: [<PyLabel at 0x7fd9b43b6950>],
 50728: [<PyLabel at 0x7fd9b4128a40>],
 50128: [<PyLabel at 0x7fd9b47d2db0>],
 29567: [<PyLabel at 0x7fd9b46a5c30>],
 20878: [<PyLabel at 0x7fd9b43a32f0>],
 34455: [<PyLabel at 0x7fd9b42d9610>],
 11229: [<PyLabel at 0x7fd9b4527750>],
 14636: [<PyLabel at 0x7fd9b404ab10>],
 29602: [<PyLabel at 0x7fd9b47f2db0>],
 29450: [<PyLabel at 0x7fd9b3d8fb40>],
 22008: [<PyLabel at 0x7fd9b3f353e0>],
 16520: [<PyLabel at 0x7fd9b463efe0>],
 5187: [<PyLabel at 0x7fd9b4656cd0>],
 24023: [<PyLabel at 0x7fd9b4558b90>],
 4246: [<PyLabel at 0x7fd9b4614180>],
 34122: [<PyLabel at 0x7fd9b3f52250>],
 35475: [<PyLabel at 0x7fd9b4

In [54]:
for node_id, bag in bags.items():
	print(node_id, bag)
	for label in bag:
		print(label)
	break

13597 [<builtins.PyLabel object at 0x7fd9b46c28e0>]


In [61]:
savable_bags = {
	node_id: [
		{
			"values": label.values,
			"hidden_values": label.hidden_values,
			"path": label.path,
			"node_id": label.node_id,
		}
		for label in bag
	]
	for node_id, bag in bags.items()
}

savable_bags

{13597: [{'values': [2098784],
   'hidden_values': None,
   'path': [0,
    1,
    2,
    50,
    28145,
    38322,
    51940,
    51939,
    51938,
    12375,
    12374,
    12373,
    12372,
    51936,
    51935,
    1101,
    496,
    51934,
    51933,
    51932,
    25336,
    24809,
    32146,
    37968,
    51931,
    51930,
    51929,
    51928,
    31526,
    51927,
    51926,
    51925,
    20871,
    51924,
    51923,
    51922,
    51921,
    51920,
    16367,
    16366,
    38449,
    38450,
    38451,
    38452,
    38453,
    38454,
    38455,
    38456,
    9589,
    9590,
    9591,
    9592,
    9593,
    9594,
    9595,
    46376,
    9575,
    9576,
    9577,
    9578,
    9579,
    9580,
    9581,
    24582,
    24551,
    24555,
    24546,
    24547,
    24548,
    24549,
    440,
    19225,
    19226,
    19227,
    436,
    452,
    19228,
    19229,
    6360,
    6380,
    7310,
    6381,
    19230,
    27937,
    1055,
    1054,
    1053,
    7534,
    7533,
   

In [30]:
storage.read_any_dict("jk")

FileNotFoundError: [Errno 2] No such file or directory: 'jk'